In [66]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.probability import FreqDist
import re
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import *
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamroth/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
path = os.getcwd()
path

'/Users/adamroth/Documents/flat_fe_work/mod4/week13/NLP---Twitter-Sentiment'

### basic info

In [8]:
raw_data = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding = 'latin1')

In [10]:
raw_data.sample(15)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
5623,RT @mention CNN to launch &quot;TV everywhere&...,NaN,No emotion toward brand or product
321,If you're at #SXSW : meet us at Fado. We will ...,NaN,No emotion toward brand or product
4259,So sad I'm missing the iPad 2 launch at *South...,iPad,Positive emotion
3193,@mention Free #SXSW Featured Artists sampler o...,NaN,No emotion toward brand or product
6117,RT @mention il CEO di flipboard annuncia la ve...,NaN,No emotion toward brand or product
1077,#Apple is setting up a pop-up store in downtow...,Apple,Positive emotion
5671,RT @mention deviantART buys 3 iPad 2's in Aust...,iPad,Positive emotion
4650,Google denies it will launch major new social ...,Other Google product or service,Positive emotion
6162,RT @mention iPhone just auto-corrected &quot;m...,NaN,No emotion toward brand or product
5763,"RT @mention Gearing up to make a splash, @ment...",NaN,No emotion toward brand or product


In [23]:
raw_data['emotion_in_tweet_is_directed_at'].sample(10)

7532    iPad
640      NaN
2848     NaN
8167     NaN
2873     NaN
3979     NaN
3328     NaN
5965     NaN
6144     NaN
3152    iPad
Name: emotion_in_tweet_is_directed_at, dtype: object

In [16]:
raw_data['emotion_in_tweet_is_directed_at'].unique()

array(['iPhone', 'iPad or iPhone App', 'iPad', 'Google', nan, 'Android',
       'Apple', 'Android App', 'Other Google product or service',
       'Other Apple product or service'], dtype=object)

In [24]:
raw_data.isna()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
9088,False,False,False
9089,False,True,False
9090,False,True,False
9091,False,True,False


In [35]:
raw_data.dtypes

tweet_text                                            object
emotion_in_tweet_is_directed_at                       object
is_there_an_emotion_directed_at_a_brand_or_product    object
dtype: object

In [25]:
raw_data['emotion_in_tweet_is_directed_at'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [ ]:
#lots of nan's in this column, shouldn't matter while training model though

In [26]:
raw_data['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [ ]:
#not many negatives, might need to mess with that, could use something like smote, might remove all from 'I cant tell'

### Tokenization

In [28]:
raw_data.iloc[i].tweet_text

'.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.'

In [30]:
raw_data.iloc[0].tweet_text.split()

['.@wesley83',
 'I',
 'have',
 'a',
 '3G',
 'iPhone.',
 'After',
 '3',
 'hrs',
 'tweeting',
 'at',
 '#RISE_Austin,',
 'it',
 'was',
 'dead!',
 'I',
 'need',
 'to',
 'upgrade.',
 'Plugin',
 'stations',
 'at',
 '#SXSW.']

In [36]:
test_list = raw_data.iloc[0].tweet_text.split()
type(test_list)

list

In [43]:
df = raw_data
df['split_text'] = 0
df.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product', 'split_text'],
      dtype='object')

In [61]:
raw_data['tweet_text'][5]

'@teachntech00 New iPad Apps For #SpeechTherapy And Communication Are Showcased At The #SXSW Conference http://ht.ly/49n4M #iear #edchat #asd'

In [64]:
for i in range(0,len(raw_data)):
    text = raw_data['tweet_text'][i]
    split_text = str(text).split()
    df['split_text'][i] = split_text
    
    



/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [65]:
df['split_text'][4]

['@sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 '#SXSW:',
 'Marissa',
 'Mayer',
 '(Google),',
 'Tim',
 "O'Reilly",
 '(tech',
 'books/conferences)',
 '&amp;',
 'Matt',
 'Mullenweg',
 '(Wordpress)']

In [74]:
import string
punctuation = string.punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [84]:
def clean_punct(string):

    clean_text_list = [''.join(ch for ch in s if ch not in punctuation) for s in string]
    
    #clean_text = ""
    
    #return (clean_text.join(clean_text_list))
    return clean_text_list
#cleans up punctuation

In [90]:
string = "@sxtxstate"
clean_punct(string)
print(clean_punct(string))

['', 's', 'x', 't', 'x', 's', 't', 'a', 't', 'e']


In [85]:
clean_punct(df['split_text'][4])

['sxtxstate',
 'great',
 'stuff',
 'on',
 'Fri',
 'SXSW',
 'Marissa',
 'Mayer',
 'Google',
 'Tim',
 'OReilly',
 'tech',
 'booksconferences',
 'amp',
 'Matt',
 'Mullenweg',
 'Wordpress']

In [89]:
# def clean_column(df['column']):
#     for i in range(0, len(df)):
#         thing = df['column'][i]
#         clean_string = str(thing)
#         return 

SyntaxError: invalid syntax (<ipython-input-89-22c6310a1aa4>, line 1)

In [91]:
df['no_punct'] = 0

In [92]:
for i in range(0, len(df)):
    has_punct = df['split_text'][i]
    no_punct = clean_punct(has_punct)
    df['no_punct'][i] = no_punct

/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [93]:
df['no_punct'][1]

['jessedee',
 'Know',
 'about',
 'fludapp',
 '',
 'Awesome',
 'iPadiPhone',
 'app',
 'that',
 'youll',
 'likely',
 'appreciate',
 'for',
 'its',
 'design',
 'Also',
 'theyre',
 'giving',
 'free',
 'Ts',
 'at',
 'SXSW']